In [1]:
from google.adk.agents import Agent
from ollama_wrapper import OllamaLLM

from prompt import TRAVEL_ADVISOR_DESCRIPTION, TRAVEL_ADVISOR_INSTRUCTION
from google.adk.tools.tool_context import ToolContext
from google.adk.sessions import DatabaseSessionService
from google.adk.artifacts import InMemoryArtifactService

MODEL = OllamaLLM(model="ollama/gpt-oss:latest").googleAdk()

sesssion_service = DatabaseSessionService(db_url="sqlite:///./session.db")

session = await sesssion_service.create_session(
    app_name="travel_advisor", user_id="user_123123", state={"user_name": "keonoh"}
)

in_memory_service = InMemoryArtifactService()


async def get_weather(tool_context: ToolContext, location: str):
    """Get current weather information for a location."""
    # Dummy implementation - returns mock data
    return {
        "location": location,
        "temperature": "22°C",
        "condition": "Partly cloudy",
        "humidity": "65%",
        "wind": "12 km/h",
        "forecast": "Mild weather with occasional clouds expected throughout the day",
    }


async def get_exchange_rate(
    tool_context: ToolContext, from_currency: str, to_currency: str, amount: float
):
    """Get exchange rate between two currencies.
    Args should always be from_currency str, to_currency str, amount flot
    """
    # Dummy implementation - returns mock data
    mock_rates = {
        ("USD", "EUR"): 0.92,
        ("USD", "GBP"): 0.79,
        ("USD", "JPY"): 149.50,
        ("USD", "KRW"): 1325.00,
        ("EUR", "USD"): 1.09,
        ("EUR", "GBP"): 0.86,
        ("GBP", "USD"): 1.27,
        ("JPY", "USD"): 0.0067,
        ("KRW", "USD"): 0.00075,
    }

    rate = mock_rates.get((from_currency, to_currency), 1.0)
    converted_amount = amount * rate

    return {
        "from_currency": from_currency,
        "to_currency": to_currency,
        "amount": amount,
        "exchange_rate": rate,
        "converted_amount": converted_amount,
        "timestamp": "2024-03-15 10:30:00 UTC",
    }


async def get_local_attractions(
    tool_context: ToolContext, location: str, category: str = "all"
):
    """Get popular attractions and points of interest for a location."""
    # Dummy implementation - returns mock data
    attractions = {
        "Paris": [
            {
                "name": "Eiffel Tower",
                "type": "landmark",
                "rating": 4.8,
                "description": "Iconic iron lattice tower",
            },
            {
                "name": "Louvre Museum",
                "type": "museum",
                "rating": 4.7,
                "description": "World's largest art museum",
            },
            {
                "name": "Arc de Triomphe",
                "type": "monument",
                "rating": 4.6,
                "description": "Historic triumphal arch",
            },
            {
                "name": "Notre-Dame",
                "type": "cathedral",
                "rating": 4.5,
                "description": "Medieval Catholic cathedral",
            },
            {
                "name": "Sacré-Cœur",
                "type": "basilica",
                "rating": 4.4,
                "description": "Romano-Byzantine basilica",
            },
        ],
        "Tokyo": [
            {
                "name": "Tokyo Tower",
                "type": "landmark",
                "rating": 4.5,
                "description": "Communications and observation tower",
            },
            {
                "name": "Senso-ji",
                "type": "temple",
                "rating": 4.6,
                "description": "Ancient Buddhist temple",
            },
            {
                "name": "Shibuya Crossing",
                "type": "landmark",
                "rating": 4.4,
                "description": "Busiest pedestrian crossing",
            },
            {
                "name": "Meiji Shrine",
                "type": "shrine",
                "rating": 4.7,
                "description": "Shinto shrine dedicated to Emperor Meiji",
            },
            {
                "name": "Tokyo Skytree",
                "type": "tower",
                "rating": 4.6,
                "description": "Broadcasting and observation tower",
            },
        ],
        "default": [
            {
                "name": "City Center",
                "type": "area",
                "rating": 4.2,
                "description": "Main downtown area",
            },
            {
                "name": "Historical Museum",
                "type": "museum",
                "rating": 4.3,
                "description": "Local history and culture",
            },
            {
                "name": "Central Park",
                "type": "park",
                "rating": 4.1,
                "description": "Main public park",
            },
            {
                "name": "Old Town",
                "type": "district",
                "rating": 4.4,
                "description": "Historic district with traditional architecture",
            },
            {
                "name": "Local Market",
                "type": "market",
                "rating": 4.0,
                "description": "Traditional local marketplace",
            },
        ],
    }

    location_attractions = attractions.get(location, attractions["default"])

    if category != "all":
        location_attractions = [
            a for a in location_attractions if a["type"] == category
        ]

    return {
        "location": location,
        "category": category,
        "attractions": location_attractions,
        "total_count": len(location_attractions),
    }


travel_advisor_agent = Agent(
    name="TravelAdvisorAgent",
    description=TRAVEL_ADVISOR_DESCRIPTION,
    instruction=TRAVEL_ADVISOR_INSTRUCTION,
    tools=[
        get_weather,
        get_exchange_rate,
        get_local_attractions,
    ],
    output_key="travel_output",
    model=MODEL,
)

/Users/keonoh/Repositories/ai-agents/google-adk/travel-advisor/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
from google.adk.runners import Runner
from google.genai import types


runner = Runner(
    agent=travel_advisor_agent,
    session_service=sesssion_service,
    app_name="travel_advisor",
    artifact_service=in_memory_service,
)

message = types.Content(
    role="user",
    parts=[types.Part(text="I am going to Malaysiz, tell me all about it")],
)

async for event in runner.run_async(
    user_id="user_123123",
    session_id=session.id,
    new_message=message,
):
    if event.is_final_response():
        print(event.content)

    else:
        print(event.get_function_calls())
        print(event.get_function_responses())

App name mismatch detected. The runner is configured with app name "travel_advisor", but the root agent was loaded from "/Users/keonoh/Repositories/ai-agents/google-adk/travel-advisor/.venv/lib/python3.14/site-packages/google/adk/agents", which implies app name "agents".


[FunctionCall(
  args={
    'location': 'Kuala Lumpur'
  },
  id='call_ha8y0dhr',
  name='get_weather'
)]
[]
[]
[FunctionResponse(
  id='call_ha8y0dhr',
  name='get_weather',
  response=<dict len=6>
)]
[FunctionCall(
  args={
    'category': 'Culture',
    'location': 'Kuala Lumpur'
  },
  id='call_22423vcg',
  name='get_local_attractions'
)]
[]
[]
[FunctionResponse(
  id='call_22423vcg',
  name='get_local_attractions',
  response={
    'attractions': [],
    'category': 'Culture',
    'location': 'Kuala Lumpur',
    'total_count': 0
  }
)]
parts=[Part(
  text="""Hey Keonoh!  
You’re headed to Malaysia—a vibrant, culturally rich country that blends bustling modern cities with lush rainforests and beautiful islands. Here’s a quick, all‑in‑one guide to help you plan your adventure.

---

## 1. Quick Snapshot of Malaysia

| Item | Detail |
|------|--------|
| **Capital** | Kuala Penang (official) | **Major city** | Kuala Lumpur |
| **Population** | ~32 million |
| **Official Language** | 